In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 범주형 데이터

In [3]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
pd.unique(values)
pd.value_counts(values)

apple     6
orange    2
dtype: int64

### ```.take()``` : Series 내에 저장된 원래 문자열 반환

In [8]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
print(values)

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64


In [9]:
print(dim)

0     apple
1    orange
dtype: object


In [10]:
print(dim.take(values))

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object


### 정수 기반의 범주형 데이터를 표현하는 Categorical 형 데이터

In [11]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [12]:
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size = N),
                   'weight': np.random.uniform(0, 4, size = N)},
                   columns = ['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,6,0.231862
1,1,orange,5,3.414720
2,2,apple,11,1.801193
3,3,apple,13,1.418625
4,4,apple,12,0.154028
5,5,orange,3,3.924177
6,6,apple,9,1.552298
7,7,apple,13,0.106345


- **```.astype('category')```**로 범주형 데이터로 변경

In [13]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [15]:
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

#### Categorical 객체는 categories와 code 속성을 가진다

In [17]:
c.categories, c.codes

(Index(['apple', 'orange'], dtype='object'),
 array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8))

- 데이터 프레임의 열을 범주형으로 변경

In [18]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

- **파이썬 열거형에서 pandas.Categorical형을 직접 생성하는 것도 가능**

In [19]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

### ```.from_codes(codes, categories)``` : 기존에 정의된 범주와 범주 코드가 있을 시, 범주형 데이터를 생성

In [20]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

In [21]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

- **```ordered = True```**로 순서형 변수 설정

In [22]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered = True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### ```.as_ordered()``` : 순서형 변수로 설정

In [23]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### 
### Categorical 연산

In [24]:
np.random.seed(12345)

In [26]:
draws = np.random.randn(1000)
draws[:5]

array([-0.98350472,  0.93094376, -0.81167564, -1.83015626, -0.13873015])

### ```pd.qcut(data, n, labels, ...)``` : 데이터를 n분위로 나눔

In [27]:
bins = pd.qcut(draws, 4)
bins

[(-3.746, -0.623], (0.677, 3.26], (-3.746, -0.623], (-3.746, -0.623], (-0.623, -0.0206], ..., (-3.746, -0.623], (-0.623, -0.0206], (-0.623, -0.0206], (-3.746, -0.623], (0.677, 3.26]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.746, -0.623] < (-0.623, -0.0206] < (-0.0206, 0.677] < (0.677, 3.26]]

- 각 분위의 label을 설정

In [28]:
bins = pd.qcut(draws, 4, labels = ['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q1', 'Q4', 'Q1', 'Q1', 'Q2', ..., 'Q1', 'Q2', 'Q2', 'Q1', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [29]:
bins.codes[:10]

array([0, 3, 0, 0, 1, 2, 2, 1, 3, 2], dtype=int8)

- bins에 이름을 붙이고 나면, 데이터의 시작값과 끝값에 대한 정보를 포함하지 않으므로, groupby를 이용해서 요약 통계 계산

In [33]:
bins = pd.Series(bins, name = 'quartile')
bins.head()

0    Q1
1    Q4
2    Q1
3    Q1
4    Q2
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [30]:
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

,quartile,count,min,max
0,Q1,250,-3.745356,-0.627426
1,Q2,250,-0.621249,-0.021299
2,Q3,250,-0.019941,0.677246
3,Q4,250,0.677263,3.260383


In [31]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

### 
### categorical을 이용한 성능 개선

In [36]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [39]:
labels.head()

0    foo
1    bar
2    baz
3    qux
4    foo
dtype: object

- labels를 categorical로 변환

In [40]:
categories = labels.astype('category')
categories.head()

0    foo
1    bar
2    baz
3    qux
4    foo
dtype: category
Categories (4, object): ['bar', 'baz', 'foo', 'qux']

- **categories가 labels에 비해 훨씬 더 적은 메모리를 사용하는 것을 확인 가능**

In [41]:
labels.memory_usage()

80000128

In [43]:
categories.memory_usage()

10000332

In [42]:
%time _ = labels.astype('category')

CPU times: total: 734 ms
Wall time: 894 ms


### 
### Categorical 메서드
|  |  |
| -- | -- |
| .add_categories() | 기존 카테고리 끝에 새로운 카테고리를 추가 |
| .as_ordered() | 카테고리가 순서를 가지도록 함 |
| .as_unordered() | 카테고리가 순서를 가지지 않도록 한다 |
| .remove_categories() | 카테고리를 제거. 해당 카테고리에 속한 값들은 null로 설정 |
| .remove_unused_categories() | 데이터에서 관측되지 않는 카테고리를 삭제 |
| .rename_categories() | 카테고리 이름을 지정한 이름으로 변경 |
| .reorder_categories() | rename_categories와 유사하지만 새로운 카테고리가 순서를 가지도록 함 |
| .set_categories | 카테고리를 지정한 새로운 카테고리로 변경. 카테고리 추가나 삭제가 가능 |

#### ```.codes()``` : 범주형 자료의 인덱스 확인
#### ```.categories()``` : 범주형 자료의 범주 확인
#### ```.value_counts()``` : 범주형 자료의 빈도 확인

In [44]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [45]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [46]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [51]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

#### ```.set_categories('categories')``` : 범주 종류 변경

In [47]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [52]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
dtype: int64

#### ```.remove_unused_categories()``` : 사용하지 않는 범주를 제거 (빈도가 0인 범주 제거)

In [53]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [54]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

In [56]:
cat_s3.value_counts()

a    2
b    2
c    0
d    0
dtype: int64

### 
### 모형 생성을 위한 가변수 (더미변수, dummy variable) 생성
### ```pd.get_dummies()```

In [58]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype = 'category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [59]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


### 
### 고급 GroupBy

In [66]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df.head()

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0


- key에 따른 그룹의 평균

In [73]:
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

- 'key'에 따른 그룹의 평균값으로 값을 변경
### ```.transform(lambda x: ...)```

In [68]:
g.transform(lambda x : x.mean())
# g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

```Python
g.transform(lambda x : x * 2) # 각 그룹에 모두 2를 곱함
g.transform(lambda x : x.rank(ascending = False)) # 각 그룹에 대해 내림차순으로 순위 계산
```

- 정규화

In [74]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [75]:
g.transform(normalize)
# g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [78]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### 
### 시계열 그룹 리샘플링

In [88]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq = '1min', periods = N)

df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})

In [90]:
df.head(5)

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4


In [84]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


### 
### 메서드 연결 기법

#### 데이터셋을 여러 차례 변형해야 하는 경우, 분석에는 필요 없는 임시 변수를 계속 생성하는 상황 발생

```Python
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
```

```Python
# 실용적이지 않은 방법
df2 = df.copy()
df2['k'] = v
```

### ```데이터프레임.assign()``` : 객체를 변경하는 대신 값 대입이 완료된 새로운 데이터 프레임을 반환
- ```.assign()```을 이용하면 메서드를 연결해서 사용할 수 있다.
- R의 ```mutate()```와 유사

```Python
# 실용적인 방법
df2 = df.assign(k = v)
```

```Python
result = (df2.assign(col1_demeaned = df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())
```

- **메서드를 연결해서 사용할 때 주의해야 할 점은 임시 객체를 참조해야 할 경우가 있을 수도 있다는 것**

```Python
df = load_data()
df2 = df[df['col2'] < 0]
```

- 위 코드는 다음과 같이 수정 가능

```Python
df = (load_data()
      [lambda x: x['col2'] < 0])
```

- 전체 코드를 하나의 메서드 연결 표현으로 작성

```Python
result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned = lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())
```

### 
### pipe 메서드

### ```.pipe()```

```Python
a = f(df, arg1 = v1)
b = g(a, v2, arg3 = v3)
c = h(b, arg4 = v4)
```

- pipe 이용

```Python
result = (df.pipe(f, arg1 = v1)
          .pipe(g, v2, arg3 = v3)
          .pipe(h, arg4 = v4))
```

#### ```f(df)```와 df.pipe(f)는 동일
- pipe를 이용한 유용한 패턴 중 하나는 일련의 연산을 재사용 가능한 함수로 일반화 하는 것

- 열에서 그룹 평균을 빼는 과정

```Python
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
```

- **pipe와 메서드 연결을 적용해서 수정**

```Python
# pipe와 메서드 연결을 적용해서 수정

def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
```

```Python
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))
```